In [ ]:
# Load .mat files into python

import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta


In [ ]:
# Experiment name for output plots
# expt_name = 'low_error_core' 
# expt_name = 'fp_precip_core' 
expt_name = 'multivar_core' 

# List of experiment names
# experiment_names = ['OLv7_M36', 'DAv7_M36_ASCAT_type_2', 'DAv7_M36_ASCAT_type_2_le', 'DAv7_M36']
# experiment_names = ['OLv7_M36', 'OLv7_M36_ascat_fp_precip', 'DAv7_M36_ASCAT_type_2', 'DAv7_M36_ASCAT_type_2_fp_precip', 'DAv7_M36']
experiment_names = ['OLv7_M36', 'DAv7_M36_ASCAT_type_2', 'DAv7_M36', 'smap_ascat_multivariate_test']

# insitu_tag plus details of timestep and number of years analysed in matlab file name
insitu_tag = '_CalVal_M33_SM_3h__6yr'

# Create MATLAB file names based on experiment names
matlab_files = [f'../test_data/SMAP/{name}'+ insitu_tag +'_raw_timeseries.mat' for name in experiment_names]

# Read the first MATLAB file to get the shape of LDAS_sm_org
first_file = matlab_files[0]
mat_contents = sio.loadmat(first_file)
shape = mat_contents['LDAS_sm_org'].shape

# Create empty array with shape of LDAS_sm_org and number of experiments
num_exp = len(experiment_names)
LDAS_sm_org = np.zeros(shape + (num_exp,))

# Read data from the first file
LDAS_sm_org[:, :, :, 0] = mat_contents['LDAS_sm_org']

# Loop over the remaining files and read the data
for i in range(1, num_exp):
    file = matlab_files[i]
    mat_contents = sio.loadmat(file)
    LDAS_sm_org[:, :, :, i] = mat_contents['LDAS_sm_org']

In [ ]:
# Print keys
print(mat_contents.keys())

In [ ]:

# Get information about date_time_vec
print(mat_contents['date_time_vec'].shape)
print(mat_contents['date_time_vec'].dtype)
# Read data
date_time_vec = mat_contents['date_time_vec']

# Get information about INSITU_sm
print(mat_contents['INSITU_sm'].shape)
print(mat_contents['INSITU_sm'].dtype)
# Read data
INSITU_sm = mat_contents['INSITU_sm']

# Get information about INSITU_id
print(mat_contents['INSITU_id'].shape)
print(mat_contents['INSITU_id'].dtype)
# Read data
INSITU_id = mat_contents['INSITU_id']

id_list = INSITU_id[0].tolist()
id_list = [str(id[0]) for id in id_list]

In [ ]:
# Get info about ubRMSE
print(LDAS_sm_org.shape)
print(LDAS_sm_org.dtype)

num_sites = LDAS_sm_org.shape[2]
print("num_sites = ", num_sites)
num_depths = LDAS_sm_org.shape[1]
print("num_depths = ", num_depths)
num_expts = LDAS_sm_org.shape[3]
print("num_expts = ", num_expts)



In [ ]:
# Extract the date and time information from date_time_vec
date_info = mat_contents['date_time_vec'][0]
years = date_info['year'].astype(int)
months = date_info['month'].astype(int)
days = date_info['day'].astype(int)
hours = date_info['hour'].astype(int)
minutes = date_info['min'].astype(int)
seconds = date_info['sec'].astype(int)

# Create a list of datetime objects representing the time steps
time_steps = []
for i in range(len(years)):
    dt = datetime(years[i], months[i], days[i], hours[i], minutes[i], seconds[i])
    time_steps.append(dt)


In [ ]:
# Calculate mean daily values for each site and soil moisture type
mean_daily_values_surface_i = [[] for _ in range(num_sites)]  # List to store mean daily values for surface soil moisture
mean_daily_values_root_i = [[] for _ in range(num_sites)]  # List to store mean daily values for root zone soil moisture
dates = []
current_day = time_steps[0].date()
daily_values_surface_i = [[] for _ in range(num_sites)]  # List to store daily values for surface soil moisture
daily_values_root_i = [[] for _ in range(num_sites)]  # List to store daily values for root zone soil moisture

for i in range(len(time_steps)):
    if time_steps[i].date() == current_day:
        for site in range(num_sites):
            daily_values_surface_i[site].append(INSITU_sm[i, 0, site])
            daily_values_root_i[site].append(INSITU_sm[i, 1, site])
    else:
        for site in range(num_sites):
            mean_daily_values_surface_i[site].append(np.mean(daily_values_surface_i[site], axis=0))
            mean_daily_values_root_i[site].append(np.mean(daily_values_root_i[site], axis=0))
        dates.append(current_day)
        daily_values_surface_i = [[] for _ in range(num_sites)]
        daily_values_root_i = [[] for _ in range(num_sites)]
        current_day = time_steps[i].date()

# Add the mean values for the last day
for site in range(num_sites):
    mean_daily_values_surface_i[site].append(np.mean(daily_values_surface_i[site], axis=0))
    mean_daily_values_root_i[site].append(np.mean(daily_values_root_i[site], axis=0))
dates.append(current_day)

# Convert dates to datetime objects
dates = [datetime.strptime(date.strftime('%Y-%m-%d'), '%Y-%m-%d').date() for date in dates]


In [ ]:

# Create 18 separate figures
for site in range(num_sites):
    fig, axs = plt.subplots(2, 1, figsize=(12, 6))
    fig.suptitle(id_list[site])

    # Plot surface values
    axs[0].plot(dates, mean_daily_values_surface_i[site], linewidth=0.5)
    axs[0].set_ylabel('Mean Surface Soil Moisture')

    # Plot root zone values
    axs[1].plot(dates, mean_daily_values_root_i[site], linewidth=0.5)
    axs[1].set_xlabel('Date')
    axs[1].set_ylabel('Mean Root Zone Soil Moisture')

    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# Calculate mean daily values for each depth, site, and expt
mean_daily_values_surface = [[[] for _ in range(num_expts)] for _ in range(num_sites)]  # List to store mean daily values for surface soil moisture
mean_daily_values_root = [[[] for _ in range(num_expts)] for _ in range(num_sites)]  # List to store mean daily values for root zone soil moisture
dates = []
current_day = time_steps[0].date()
daily_values_surface = [[[] for _ in range(num_expts)] for _ in range(num_sites)]  # List to store daily values for surface soil moisture
daily_values_root = [[[] for _ in range(num_expts)] for _ in range(num_sites)]  # List to store daily values for root zone soil moisture

for i in range(len(time_steps)):
    if time_steps[i].date() == current_day:
        for site in range(num_sites):
            for expt in range(num_expts):
                daily_values_surface[site][expt].append(LDAS_sm_org[i, 0, site, expt])
                daily_values_root[site][expt].append(LDAS_sm_org[i, 1, site, expt])
    else:
        for site in range(num_sites):
            for expt in range(num_expts):
                mean_daily_values_surface[site][expt].append(np.mean(daily_values_surface[site][expt], axis=0))
                mean_daily_values_root[site][expt].append(np.mean(daily_values_root[site][expt], axis=0))
        dates.append(current_day)
        daily_values_surface = [[[] for _ in range(num_expts)] for _ in range(num_sites)]
        daily_values_root = [[[] for _ in range(num_expts)] for _ in range(num_sites)]
        current_day = time_steps[i].date()

# Add the mean values for the last day
for site in range(num_sites):
    for expt in range(num_expts):
        mean_daily_values_surface[site][expt].append(np.mean(daily_values_surface[site][expt], axis=0))
        mean_daily_values_root[site][expt].append(np.mean(daily_values_root[site][expt], axis=0))
dates.append(current_day)

# Convert dates to datetime objects
dates = [datetime.strptime(date.strftime('%Y-%m-%d'), '%Y-%m-%d').date() for date in dates]

In [ ]:
# Create 18 separate plots

expt_labels = experiment_names

for site in range(num_sites):
    fig, axs = plt.subplots(2, 1, figsize=(16, 8))
    fig.suptitle(id_list[site])

    # Plot surface values
    axs[0].set_title("Surface")
    axs[0].plot(dates, mean_daily_values_surface_i[site], color='black', label='In Situ',  linewidth=0.5)
    for expt in range(num_expts):
        axs[0].plot(dates, mean_daily_values_surface[site][expt], label=expt_labels[expt], linewidth=0.5)
    axs[0].set_ylabel('Mean Soil Moisture')
    axs[0].legend()

    # Plot root zone values
    axs[1].set_title("Root Zone")
    axs[1].plot(dates, mean_daily_values_root_i[site], color='black', label='In Situ',  linewidth=0.5)
    for expt in range(num_expts):
        axs[1].plot(dates, mean_daily_values_root[site][expt], label=expt_labels[expt], linewidth=0.5)
    axs[1].set_xlabel('Date')
    axs[1].set_ylabel('Mean Soil Moisture')
    axs[1].legend()

    plt.xticks(rotation=45)
    plt.tight_layout()

    plt.savefig(expt_name + "_" + id_list[site] + "_ts.png")
    plt.show()

In [ ]:
# Calculate mean monthly values for each site and soil moisture type
mean_monthly_values_surface_i = [[] for _ in range(num_sites)]  # List to store mean monthly values for surface soil moisture
mean_monthly_values_root_i = [[] for _ in range(num_sites)]  # List to store mean monthly values for root zone soil moisture
monthly_dates = []
current_month = time_steps[0].date().replace(day=1)
monthly_values_surface_i = [[] for _ in range(num_sites)]  # List to store monthly values for surface soil moisture
monthly_values_root_i = [[] for _ in range(num_sites)]  # List to store monthly values for root zone soil moisture

for i in range(len(time_steps)):
    if time_steps[i].date().replace(day=1) == current_month:
        for site in range(num_sites):
            monthly_values_surface_i[site].append(INSITU_sm[i, 0, site])
            monthly_values_root_i[site].append(INSITU_sm[i, 1, site])
    else:
        for site in range(num_sites):
            mean_monthly_values_surface_i[site].append(np.nanmean(monthly_values_surface_i[site], axis=0))
            mean_monthly_values_root_i[site].append(np.nanmean(monthly_values_root_i[site], axis=0))
        monthly_dates.append(current_month)
        monthly_values_surface_i = [[] for _ in range(num_sites)]
        monthly_values_root_i = [[] for _ in range(num_sites)]
        current_month = time_steps[i].date().replace(day=1)

# Add the mean values for the last month
for site in range(num_sites):
    mean_monthly_values_surface_i[site].append(np.nanmean(monthly_values_surface_i[site], axis=0))
    mean_monthly_values_root_i[site].append(np.nanmean(monthly_values_root_i[site], axis=0))
monthly_dates.append(current_month)

# Convert dates to datetime objects
monthly_dates = [datetime.strptime(date.strftime('%Y-%m-%d'), '%Y-%m-%d').date() for date in monthly_dates]

In [ ]:
# Calculate mean monthly values for each depth, site, and expt
mean_monthly_values_surface = [[[] for _ in range(num_expts)] for _ in range(num_sites)]  # List to store mean monthly values for surface soil moisture
mean_monthly_values_root = [[[] for _ in range(num_expts)] for _ in range(num_sites)]  # List to store mean monthly values for root zone soil moisture
monthly_dates = []
current_month = time_steps[0].date().replace(day=1)
monthly_values_surface = [[[] for _ in range(num_expts)] for _ in range(num_sites)]  # List to store monthly values for surface soil moisture
monthly_values_root = [[[] for _ in range(num_expts)] for _ in range(num_sites)]  # List to store monthly values for root zone soil moisture

for i in range(len(time_steps)):
    if time_steps[i].date().replace(day=1) == current_month:
        for site in range(num_sites):
            for expt in range(num_expts):
                monthly_values_surface[site][expt].append(LDAS_sm_org[i, 0, site, expt])
                monthly_values_root[site][expt].append(LDAS_sm_org[i, 1, site, expt])
    else:
        for site in range(num_sites):
            for expt in range(num_expts):
                mean_monthly_values_surface[site][expt].append(np.mean(monthly_values_surface[site][expt], axis=0))
                mean_monthly_values_root[site][expt].append(np.mean(monthly_values_root[site][expt], axis=0))
        monthly_dates.append(current_month)
        monthly_values_surface = [[[] for _ in range(num_expts)] for _ in range(num_sites)]
        monthly_values_root = [[[] for _ in range(num_expts)] for _ in range(num_sites)]
        current_month = time_steps[i].date().replace(day=1)

# Add the mean values for the last month
for site in range(num_sites):
    for expt in range(num_expts):
        mean_monthly_values_surface[site][expt].append(np.mean(monthly_values_surface[site][expt], axis=0))
        mean_monthly_values_root[site][expt].append(np.mean(monthly_values_root[site][expt], axis=0))
monthly_dates.append(current_month)

# Convert dates to datetime objects
monthly_dates = [datetime.strptime(date.strftime('%Y-%m-%d'), '%Y-%m-%d').date() for date in monthly_dates]

In [ ]:
# Create 18 separate plots

for site in range(num_sites):
    fig, axs = plt.subplots(2, 1, figsize=(16, 8))
    fig.suptitle(id_list[site])

    # Plot surface values
    axs[0].set_title("Surface")
    axs[0].plot(monthly_dates, mean_monthly_values_surface_i[site], color='black', label='In Situ',  linewidth=0.8)
    for expt in range(num_expts):
        axs[0].plot(monthly_dates, mean_monthly_values_surface[site][expt], label=expt_labels[expt], linewidth=0.8)
    axs[0].set_ylabel('Mean Soil Moisture')
    axs[0].legend()

    # Plot root zone values
    axs[1].set_title("Root Zone")
    axs[1].plot(monthly_dates, mean_monthly_values_root_i[site], color='black', label='In Situ',  linewidth=0.8)
    for expt in range(num_expts):
        axs[1].plot(monthly_dates, mean_monthly_values_root[site][expt], label=expt_labels[expt], linewidth=0.8)
    axs[1].set_xlabel('Date')
    axs[1].set_ylabel('Mean Soil Moisture')
    axs[1].legend()

    plt.xticks(rotation=45)
    plt.tight_layout()

    plt.savefig(expt_name + "_" + id_list[site] + "_monthly_ts.png")
    plt.show()